# Real Estate Analysis

## Overview

This project is an analysis of real estate within California. The goal of the project is to be able to predict sale price based on the features of the house. 

## Business Problem

The business problem for this project is to accurately price houses for sale according to the features of the house. 

## Data Understanding

The California Real Estate dataset from Kaggle has around 35,000 records of houses within the state. There are a total of 38 columns and include data such as location, size, and amenities. 

In [1]:
#Importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns


In [2]:
df = pd.read_csv('data/RealEstate_California.csv', index_col=0) #loading the data
df.head() #Viewing the first five rows of data

,id,stateId,countyId,cityId,country,datePostedString,is_bankOwned,is_forAuction,event,time,...,parking,garageSpaces,hasGarage,levels,pool,spa,isNewConstruction,hasPetsAllowed,homeType,county
0,95717-2087851113,9,77,24895,USA,2021-01-13,0,0,Listed for sale,1.610496e+12,...,0,0.0,0,0,0,0,0,0,LOT,Placer County
1,94564-18496265,9,189,36958,USA,2021-07-12,0,0,Listed for sale,1.626048e+12,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
2,94564-18484475,9,190,36958,USA,2021-07-08,0,0,Listed for sale,1.625702e+12,...,1,2.0,1,One Story,0,0,0,0,SINGLE_FAMILY,Contra Costa County
3,94564-18494835,9,191,36958,USA,2021-07-07,0,0,Listed for sale,1.625616e+12,...,1,1.0,1,Two Story,0,1,0,0,SINGLE_FAMILY,Contra Costa County
4,94564-2069722747,9,192,36958,USA,2021-07-07,0,0,Listed for sale,1.625616e+12,...,0,0.0,0,0,0,0,0,0,LOT,Contra Costa County


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35389 entries, 0 to 35388
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  35389 non-null  object 
 1   stateId             35389 non-null  int64  
 2   countyId            35389 non-null  int64  
 3   cityId              35389 non-null  int64  
 4   country             35389 non-null  object 
 5   datePostedString    35386 non-null  object 
 6   is_bankOwned        35389 non-null  int64  
 7   is_forAuction       35389 non-null  int64  
 8   event               35389 non-null  object 
 9   time                35100 non-null  float64
 10  price               35389 non-null  float64
 11  pricePerSquareFoot  35389 non-null  float64
 12  city                35389 non-null  object 
 13  state               35389 non-null  object 
 14  yearBuilt           35389 non-null  int64  
 15  streetAddress       35389 non-null  object 
 16  zipc

In [4]:
df['streetAddress'].duplicated().sum() #Checking for duplicated addresses

4259

In [8]:
#df['streetAddress'] = df[df['streetAddress'].drop_duplicates(keep='first')]

In [ ]:
df['streetAddress'].drop_duplicates(keep='first', inplace=True, axis=0)

In [ ]:
df['streetAddress'].duplicated().sum()

In [ ]:
print(f'There are {df['streetAddress'].duplicated().sum()} duplicated values') 
#Verifying duplicated values have been dropped

In [ ]:
print(f'There are {df.duplicated().sum()} duplicated values') #Checking for any duplicated values

In [ ]:
df.drop(['id', 
         'stateId', 
         'country', 
         'cityId', 
         'countyId', 
         'time',  
         'state', 
         'currency'], 
         axis=1, 
         inplace=True) 
#Dropping variables without predictor value or redundant

In [ ]:
df.info() #looking at the overall columns/features

In [ ]:
df.isna().sum() #checking for null values

In [ ]:
df.dropna(inplace=True) #dropping null values since there aren't too many when looking at the overall dataset

In [ ]:
df.isna().sum() #verifying there's no more nulls

In [ ]:
df['year'] = pd.to_datetime(df['datePostedString']) #Converting date posted to year to observe yearly trends
df['year'] = df['year'].dt.year
df.drop(['datePostedString'], inplace=True, axis=1) #Dropping original date posted column

### Feature Analysis

In [ ]:
def feature_corr(feature):
    feature_corr = df_single['price'].corr(df_single[feature])
    return print(f'{feature.title()} correlation with popularity is {feature_corr}')

def feature_analysis(feature):
    '''Creates distplot and graph of feature and mean of price along with correlation and value counts'''
    
    feature_corr(feature)
    
    print(df_single[feature].value_counts(normalize=True))
    
    fig, ax = plt.subplots(figsize=(15,8))
    sns.distplot(df_single[feature], ax=ax)
    ax.set_title(f'{feature.title()} Distribution', fontsize=16)
    ax.set_xlabel(f'{feature.title()}', fontsize=14)
    ax.set_ylabel('Density', fontsize=14)
    
    print(df_single[feature].describe())
    
    feature_price_mean = df_single.groupby(feature)['price'].mean()
    
    fig, ax = plt.subplots(figsize=(15,8))
    ax.scatter(feature_price_mean.index, feature_price_mean.values, alpha=.3)
    ax.set_title(f'{feature.title()} vs Mean Price', fontsize=16)
    ax.set_xlabel(f'{feature.title()}', fontsize=14)
    ax.set_ylabel('Mean Price', fontsize=14)
    return plt.show();

## Single Family Home Analysis

In [ ]:
df['homeType'].value_counts()

In [ ]:
df_single = df[df['homeType'] == 'SINGLE_FAMILY'] #Filtering out the data for single family homes only
df_single['homeType'].value_counts()

#### Bank Owned

In [ ]:
df_single['is_bankOwned'].value_counts(normalize=True) #Seeing how many homes are bank owned

In [ ]:
feature_analysis('is_bankOwned')

In [ ]:
df_single = df_single[df_single['is_bankOwned'] == 0] #Filtering out houses that are bank owned
df_single['is_bankOwned'].value_counts() #Verifying no more bank owned homes

In [ ]:
df_single.drop('is_bankOwned', axis=1, inplace=True)  #Dropping the column since they're all non bank owned properties

#### Auction

In [ ]:
df_single['is_forAuction'].value_counts(normalize=True) #Seeing how many homes are for auction

In [ ]:
feature_analysis('is_forAuction')

In [ ]:
df_single = df_single[df_single['is_forAuction'] == 0] #Filtering out homes for auction
df_single['is_forAuction'].value_counts() #Verifying no more homes for auction

In [ ]:
df_single.drop('is_forAuction', axis=1, inplace=True) #Dropping the column since no homes are for auction

#### Event

In [ ]:
df_single['event'].value_counts()

In [ ]:
df_single.info()

#### Bathrooms

In [ ]:
df_single['bathrooms'].value_counts()

In [ ]:
avg_price_bath_single = df_single.groupby('bathrooms')['price'].mean().sort_values(ascending=False)
avg_price_bath_single

In [ ]:
feature_analysis('bathrooms')

In [ ]:
df_single = df_single[(df_single['bathrooms'] < 7) & (df_single['bathrooms'] != 0)] #Filtering out homes without bathrooms and more than 7 for the standard home 
df_single['bathrooms'].value_counts()

In [ ]:
feature_analysis('bathrooms')

#### Bedrooms

In [ ]:
df_single['bedrooms'].value_counts(normalize=True)

In [ ]:
avg_price_bed_single = df_single.groupby('bedrooms')['price'].mean().sort_values(ascending=False)
avg_price_bed_single

In [ ]:
feature_analysis('bedrooms')

In [ ]:
df_single = df_single[df_single['bedrooms'] <= 7] #Filtering out houses with more than 7 bedrooms for a standard home
df_single['bedrooms'].value_counts()

In [ ]:
feature_analysis('bedrooms')

In [ ]:
bedroom_outliers = df_single[(df_single['bedrooms'] == 0) & (df_single['price'] > 1000000)]
bedroom_outliers

#### Target: Popularity Analysis

In [ ]:
df['price'].describe()

In [ ]:
df['price'].value_counts(bins=8)

In [ ]:
df['price'].max()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
ax.hist(df['price'], bins=50);

In [ ]:
#Boxplot with outliers
fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(x=df['price'], showmeans=True, meanline=True);

In [ ]:
#Boxplot without outliers
fig, ax = plt.subplots(figsize=(15, 8))
sns.boxplot(x=df['price'], showfliers=False);

In [ ]:
sns.displot(df['price'], height=7, aspect=2);

In [ ]:
pop_corr = abs(df.corr()[['price']]).sort_values('price', ascending=False) #Correlation analysis against price
pop_corr

In [ ]:
fig, ax = plt.subplots(figsize=(15, 8))
sns.heatmap(pop_corr, ax=ax, annot=True)
ax.set_yticklabels(labels=pop_corr.index, fontsize=12);

### Feature Exploration